In [1]:
import pandas as pd
import numpy as np
# для парсинга
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [2]:
def wall_parser():
    token = '0da629350da629350da62935b60dd2bd8200da60da629355233580cc925888190d01861'
    api = 'https://api.vk.com/method/wall.get'
    version = 5.92
    domain = 'knife.media'
    offset = 0
    count = 100
    all_posts = []

    while offset < 9000:
        response = requests.get(api,
                                params = {'access_token': token,
                                          'v': version,
                                          'domain': domain,
                                          'count': count,
                                          'offset': offset}
                               )
        data = response.json()['response']['items']
        offset += count
        all_posts.extend(data)
    return all_posts

In [3]:
def filter_inf():
    filter_data_list = []
    all_posts = wall_parser()
    for post in all_posts:
        filter_data_list.append([post['id'],
                                 post['date'],
                                 post['likes']['count'],
                                 post['comments']['count'],
                                 post['reposts']['count'],
                                 post['views']['count'],
                                 post['text']])
    return filter_data_list

In [4]:
# изменить шаблон под более общий !!!!!!!!!!
def try_url(x):
    pattern = r'(https://kn.+)|(http://knf.+)'
    try:
        return re.search(pattern, x)[0]
    except:
        return ('false')
# def try_url(x):
#     pattern = r'.*(http.+)'
#     try:
#         if re.findall(pattern, x)[0]:
#             return re.findall(pattern, x)[0]
#     except:
#         return ('false')

In [5]:
# функция парсинга текста
def text_parser(url):
    try:
        html = urlopen(url).read().decode('utf-8')
        # возможно, сделать шаблон более общим
        pattern_2 = r'.+"text":"(.+)"}</script>'
        text = re.findall(pattern_2, html)[0].replace(u'\xa0', ' ')
    except:
        text = ''
    return text

In [7]:
def tags(link):
    tags_page = []
    try:
        html = urlopen(link).read().decode('utf-8')
        soup = BeautifulSoup(html,'html.parser')
        tags = soup.find_all(rel="tag")
        for tag in tags:
            tags_page.append(tag.text)
    except:
        tags_page = None
    return tags_page

In [8]:
def cleaner_text(link):
    text = text_parser(link)
    text = re.sub(r'[._^%$#!~@,-«»]+', ' ', text)
    text = re.sub('[^а-яА-Я ]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text

In [16]:
cleaner_text('https://knife.media/antiravinagar-book/')

'То что Роман Михайлов один из самых интересных современных русских писателей можно считать уже практически общепризнанным и его новая книга Антиравинагар вышедшая на днях в издательстве лишнее тому подтверждение Сам Михайлов на сегодняшний день считает это произведение своим пока шла подготовка издания он успел поставить по нему в БДТ одноименный радиоспектакль а также обычный точнее необычный спектакль Несолнечный город Сказать о чем эта книга довольно непросто Роман настолько далек от всех литературных традиций настоящего и прошлого что ему пришлось придумать собственную литературу работающую по правилам которые хорошо понимает только он сам что впрочем нисколько не мешает его все более многочисленным читателям Согласно определению автора Антиравинагар это трактат о памяти и ритуалах Он состоит из зыбких сложным образом перекликающихся друг с другом историй что не мешает им оставаться в первую очередь историями которые интересно читать В числе действующих лиц колдуны бандиты и птицы

In [ ]:
columns = ['id', 'time', 'likes', 'comments', 'reposts', 'views', 'text']
df = pd.DataFrame(filter_inf(), columns = columns)

In [9]:
df['ctr'] = (df.likes / df.views * 100).round(2)

In [ ]:
df['url'] = df.text.apply(try_url)

In [ ]:
df['year'] = pd.to_datetime(df.time, unit='s').dt.to_period('Y')
df['month'] = pd.to_datetime(df.time, unit='s').dt.to_period('M')
df['day'] = pd.to_datetime(df.time, unit='s').dt.to_period('D')

In [ ]:
df.head()

In [ ]:
filtered_df = df.query("url != 'false' and day >= '2018-01-01'")

In [ ]:
filtered_df.url.count()

In [16]:
filtered_df['tags'] = filtered_df.url.apply(tags)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
filtered_df.isna().sum()

id           0
time         0
likes        0
comments     0
reposts      0
views        0
text         0
ctr          0
url          0
year         0
month        0
day          0
tags        44
dtype: int64

In [19]:
filtered_df.dropna(inplace=True)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
filtered_df.head()

,id,time,likes,comments,reposts,views,text,ctr,url,year,month,day,tags
1,319627,1604761803,44,12,3,17592,"Алкотестеры можно обмануть, а вот походка выда...",0.25,https://knife.media/smartphone-as-alcodetector/,2020,2020-11,2020-11-07,"[алкоголь, технологии]"
2,319586,1604751003,54,25,10,12168,Жизнь молодого государства после Февральской р...,0.44,https://knife.media/october-revolution/,2020,2020-11,2020-11-07,"[история, города, политика, россия]"
3,319578,1604748303,310,28,25,18554,"Вещество, содержащееся в галлюциногенных гриба...",1.67,https://knife.media/psilocybin-assisted-therapy/,2020,2020-11,2020-11-07,"[здоровье, исследование, наркотики]"
4,319544,1604736602,380,10,45,28104,"Радость от мороженого длится пару мгновений, о...",1.35,https://knife.media/hedonic-adaptation/,2020,2020-11,2020-11-07,"[мозг, наука, саморазвитие]"
5,319424,1604676602,82,28,3,15232,Тогда у человечества появятся хоть какие-то ша...,0.54,https://knife.media/less-meat/,2020,2020-11,2020-11-06,"[еда, исследование, экология]"


In [39]:
def need_tags(x):
    tags_list = ['находка', 'литература', 'психология', 'наука', 'россия']
    for i in tags_list:
        if i in x:
            return 1
        else:
            continue
    return 0

In [41]:
filtered_df.tags.apply(need_tags).sum()

4339

In [22]:
explode_tags = filtered_df.explode('tags')

In [23]:
explode_tags.head(10)

,id,time,likes,comments,reposts,views,text,ctr,url,year,month,day,tags
1,319627,1604761803,44,12,3,17592,"Алкотестеры можно обмануть, а вот походка выда...",0.25,https://knife.media/smartphone-as-alcodetector/,2020,2020-11,2020-11-07,алкоголь
1,319627,1604761803,44,12,3,17592,"Алкотестеры можно обмануть, а вот походка выда...",0.25,https://knife.media/smartphone-as-alcodetector/,2020,2020-11,2020-11-07,технологии
2,319586,1604751003,54,25,10,12168,Жизнь молодого государства после Февральской р...,0.44,https://knife.media/october-revolution/,2020,2020-11,2020-11-07,история
2,319586,1604751003,54,25,10,12168,Жизнь молодого государства после Февральской р...,0.44,https://knife.media/october-revolution/,2020,2020-11,2020-11-07,города
2,319586,1604751003,54,25,10,12168,Жизнь молодого государства после Февральской р...,0.44,https://knife.media/october-revolution/,2020,2020-11,2020-11-07,политика
2,319586,1604751003,54,25,10,12168,Жизнь молодого государства после Февральской р...,0.44,https://knife.media/october-revolution/,2020,2020-11,2020-11-07,россия
3,319578,1604748303,310,28,25,18554,"Вещество, содержащееся в галлюциногенных гриба...",1.67,https://knife.media/psilocybin-assisted-therapy/,2020,2020-11,2020-11-07,здоровье
3,319578,1604748303,310,28,25,18554,"Вещество, содержащееся в галлюциногенных гриба...",1.67,https://knife.media/psilocybin-assisted-therapy/,2020,2020-11,2020-11-07,исследование
3,319578,1604748303,310,28,25,18554,"Вещество, содержащееся в галлюциногенных гриба...",1.67,https://knife.media/psilocybin-assisted-therapy/,2020,2020-11,2020-11-07,наркотики
4,319544,1604736602,380,10,45,28104,"Радость от мороженого длится пару мгновений, о...",1.35,https://knife.media/hedonic-adaptation/,2020,2020-11,2020-11-07,мозг


In [36]:
explode_tags.shape

(23556, 13)

In [24]:
explode_tags.to_csv('explode_tags.csv', index=False)

In [25]:
tags_list = ['находка', 'литература', 'психология', 'наука', 'россия']

In [28]:
df_for_classification = explode_tags.query("tags in @ tags_list")

In [31]:
df_for_classification.head()

,id,time,likes,comments,reposts,views,text,ctr,url,year,month,day,tags
2,319586,1604751003,54,25,10,12168,Жизнь молодого государства после Февральской р...,0.44,https://knife.media/october-revolution/,2020,2020-11,2020-11-07,россия
4,319544,1604736602,380,10,45,28104,"Радость от мороженого длится пару мгновений, о...",1.35,https://knife.media/hedonic-adaptation/,2020,2020-11,2020-11-07,наука
6,319419,1604675403,46,14,5,26881,Евгений «Маэстро» Дмитерко — артист милостью б...,0.17,https://knife.media/maestro-ryba/,2020,2020-11,2020-11-06,россия
8,319395,1604670363,177,2,18,12200,«Жизнь коротка. И надо уметь. Надо уметь уходи...,1.45,https://knife.media/zhvanetsky-rip/,2020,2020-11,2020-11-06,литература
8,319395,1604670363,177,2,18,12200,«Жизнь коротка. И надо уметь. Надо уметь уходи...,1.45,https://knife.media/zhvanetsky-rip/,2020,2020-11,2020-11-06,россия


In [44]:
df_for_classification.shape

(5149, 13)

In [35]:
df_for_classification.id.nunique()

4339

In [68]:
df_for_classification_target = pd.get_dummies(df_for_classification[['id', 'tags']]).groupby('id', as_index=False).sum()

In [69]:
df_for_classification_target.head()

,id,tags_литература,tags_наука,tags_находка,tags_психология,tags_россия
0,145781,0,0,0,1,0
1,145784,0,1,0,0,0
2,145792,0,1,0,0,0
3,145804,0,1,0,0,0
4,145835,0,1,0,0,0


In [70]:
df_for_classification_target.sum()

id                 996850981
tags_литература          706
tags_наука               883
tags_находка             494
tags_психология          865
tags_россия             2201
dtype: int64

In [71]:
df_for_classification_target.shape

(4339, 6)

In [72]:
df_for_classification_target.to_csv('tags_for_fit.csv', index=False)

In [48]:
df_for_classification_feature = filtered_df.query("id in @df_for_classification_target.id")[['id', 'ctr', 'url']]

In [49]:
df_for_classification_feature.head()

,id,ctr,url
2,319586,0.44,https://knife.media/october-revolution/
4,319544,1.35,https://knife.media/hedonic-adaptation/
6,319419,0.17,https://knife.media/maestro-ryba/
8,319395,1.45,https://knife.media/zhvanetsky-rip/
9,319368,0.27,https://knife.media/cancer-therapy-study/


In [50]:
df_for_classification_feature.shape

(4339, 3)

In [60]:
df_for_classification_feature['text'] = df_for_classification_feature.url.apply(cleaner_text)

In [61]:
df_for_classification_feature.head()

,id,ctr,url,text
2,319586,0.44,https://knife.media/october-revolution/,Ровно года назад ноября года октября по старом...
4,319544,1.35,https://knife.media/hedonic-adaptation/,Когда происходит что то хорошее мы старательно...
6,319419,0.17,https://knife.media/maestro-ryba/,Евгений Маэстро Дмитерко недавно вспыхнувшая з...
8,319395,1.45,https://knife.media/zhvanetsky-rip/,Умер писатель сатирик Михаил Жванецкий Ему был...
9,319368,0.27,https://knife.media/cancer-therapy-study/,Ученые Института молекулярной и клеточной биол...


In [62]:
df_for_classification_feature.to_csv('texts.csv', index=False)

In [7]:
df = pd.read_csv('texts.csv')

In [9]:
df.url.nunique()

3759